# 05 - Dashboard Development

## Introduction

This notebook focuses on building an interactive dashboard to help decision-makers at Corporación Favorita visually explore sales forecasts, performance trends, and stock planning indicators across stores and product families.

### Objectives 

The goal is to translate our forecasting results and business insights into a user-friendly interface that supports:

- **Inventory planning**: Highlighting high-risk product/store combinations
- **Promotion effectiveness**: Visualizing the impact of promotions on sales
- **Forecast reliability**: Displaying error distributions and confidence levels
- **Geographic insight**: Mapping store performance by city or region

The dashboard aims to provide **actionable analytics** that assist planners and managers in making better stocking and replenishment decisions—quickly, and with confidence.


## 1. Import and Load Data

In [15]:
# --- Import core libraries ---
import pandas as pd
import plotly.express as px
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns

# --- Load data ---
forecast_df = pd.read_csv("../forecasts/forecast_results.csv")
actual_df = pd.read_csv("../data/train.csv")
store_meta = pd.read_csv("../data/stores.csv")

# --- Prepare date columns ---
forecast_df['ds'] = pd.to_datetime(forecast_df['ds'])
actual_df['date'] = pd.to_datetime(actual_df['date'])

# --- Streamlit page config ---
st.set_page_config(page_title="Sales Forecast Dashboard", layout="wide")
st.title("Sales Forecast Dashboard - Corporación Favorita")


2025-05-21 05:34:20.584 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 05:34:20.585 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 05:34:20.586 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

## 2. Implement Filters


Use the filters in the sidebar to explore forecasts by **store**, **product family**, and **forecast date range**.  
This interactive view helps stakeholders examine specific segments, identify periods with **promotions** or **holidays**, and assess their impact on forecast accuracy and inventory needs.


In [ ]:
# --- Sidebar Filters ---
st.sidebar.header("Filter Forecasts")

# Filter options
store_options = sorted(forecast_df['store_nbr'].unique())
family_options = sorted(forecast_df['family'].unique())
date_min = forecast_df['ds'].min()
date_max = forecast_df['ds'].max()

# Widgets
selected_store = st.sidebar.selectbox("Select Store Number", store_options)
selected_family = st.sidebar.selectbox("Select Product Family", family_options)
selected_dates = st.sidebar.date_input(
    "Select Forecast Date Range",
    value=(date_min, date_max),
    min_value=date_min,
    max_value=date_max
)

# Holiday and Promotion toggles
highlight_holidays = st.sidebar.checkbox("Highlight Holidays", value=False)
show_promotions = st.sidebar.checkbox("Show Promotions", value=False)


2025-05-21 05:34:20.594 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 05:34:20.595 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 05:34:20.598 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 05:34:20.599 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 05:34:20.599 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 05:34:20.599 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 05:34:20.600 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 05:34:20.600 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

## 3. Forecast vs. Actual Sales Plot

This chart visualizes forecasted vs. actual sales for the selected store and product family.  
It highlights how **promotions** and **holidays** may influence sales behavior—supporting smarter inventory and replenishment decisions.


In [ ]:
# --- Filter Data ---
start_date = pd.to_datetime(selected_dates[0])
end_date = pd.to_datetime(selected_dates[1])

filtered_forecast = forecast_df[
    (forecast_df['store_nbr'] == selected_store) &
    (forecast_df['family'] == selected_family) &
    (forecast_df['ds'] >= start_date) &
    (forecast_df['ds'] <= end_date)
]

actuals_df = actual_df.rename(columns={'date': 'ds'})
filtered_actuals = actuals_df[
    (actuals_df['store_nbr'] == selected_store) &
    (actuals_df['family'] == selected_family) &
    (actuals_df['ds'] >= start_date) &
    (actuals_df['ds'] <= end_date)
]

# --- Merge forecast with actuals ---
merged_df = pd.merge(filtered_forecast, filtered_actuals, on=['ds', 'store_nbr', 'family'], how='left')

# --- Plot ---
st.subheader("Forecast vs. Actual Sales")
fig, ax = plt.subplots(figsize=(12, 6))

# Forecast and actual
sns.lineplot(data=merged_df, x='ds', y='yhat', label='Forecast', ax=ax)
sns.lineplot(data=merged_df, x='ds', y='sales', label='Actual', ax=ax)

# Optionally highlight holidays
if highlight_holidays:
    holiday_dates = merged_df[merged_df['is_holiday'] == 1]['ds']
    for h_date in holiday_dates:
        ax.axvline(h_date, color='red', linestyle='--', alpha=0.3)

# Optionally show promotion flags
if show_promotions:
    promo_days = merged_df[merged_df['onpromotion'] == 1]
    sns.scatterplot(data=promo_days, x='ds', y='sales', color='green', label='Promo Day', ax=ax)

ax.set_title(f"Store {selected_store} - {selected_family}")
ax.set_xlabel("Date")
ax.set_ylabel("Sales")
plt.xticks(rotation=45)
st.pyplot(fig)


2025-05-21 05:34:20.839 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 05:34:20.840 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 05:34:20.878 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 05:34:21.170 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 05:34:21.171 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()